In [ ]:
%matplotlib inline
%load_ext autoreload
%autoreload 2
from utils.testcase_creator import *
from utils.app_creator import create_dags
from optimization.sa.task_graph import PrecedenceGraph
import networkx as nx
import main
from pathlib import Path
from input.input_parameters import InputParameters, EMode, Timeouts

# Create random application DAGs
- Clone https://github.com/perarnau/ggen
- Run vagrant container ("vagrant up")
- Map utils/apps/ folder to /apps in container
- Run "vagrant global-status" to obtain CONTAINER_ID

In [ ]:
CONTAINER_ID = "a9ff793"

In [ ]:
config.app_task_connection_probability = 0.4
config.nr_tasks = 24
config.nr_dags = (config.nr_tasks // 8) + 1
config.periods = [10000, 15000, 20000, 50000]
        
# Create set of DAGs
for i in range(10):
    create_dags(config.nr_tasks // config.nr_dags, random.randint(config.min_app_depth, config.max_app_depth), config.app_task_connection_probability, f"dag{i}", CONTAINER_ID)


In [ ]:
batch_nr = 4
cases_per_batch = 25


# Create topology
config = testcase_generation_config()
config.app_task_connection_probability = 0.4
config.nr_tasks = 24
config.nr_dags = (config.nr_tasks // 8) + 1
config.periods = [10000, 15000, 20000, 50000]
config.nr_sw = 8
config.nr_es = 16
config.link_speed = 12.5 # 100 mbit
config.connections_per_sw = config.nr_sw -1
config.stream_max_rl = 3
config.connections_per_es = 5
G, points_sw, points_es = topology_creator.generate_topology(config.nr_sw, config.nr_es, config.connections_per_sw,
                                                                 config.connections_per_es)
i = 2    
for j in range(cases_per_batch):
        if i == 0:
            config.stream_min_size = 1000
            config.stream_max_size = 1500
            config.max_task_period_percentage = 0.02
        if i == 1:
            config.stream_min_size = 1000
            config.stream_max_size = 1500
            config.max_task_period_percentage = 0.1
        if i == 3:
            config.stream_min_size = 1
            config.stream_max_size = 250
            config.max_task_period_percentage = 0.02
        if i == 2:
            config.stream_min_size = 1
            config.stream_max_size = 250
            config.max_task_period_percentage = 0.1
            

        path = create_testcase_with_topology_and_dags(config, Path(f"testcases/impact/batch{i}{j}.flex_network_description"), G, points_sw, points_es, CONTAINER_ID)
        #solution_object = main.run(InputParameters(EMode.VIEW, Timeouts(0, 0, 0), path, True, "", 8050, False, False, False))